In [3]:
### Working With Tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

## Arxiv and wikipedia Tools
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=2000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
tools = [wiki , arxiv]

In [5]:
# custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=2000 , chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents , embeddings)
retrever = vectorstore.as_retriever()
retrever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbcaaec37f0>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool

In [10]:
retriever_tool=create_retriever_tool(retrever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name


'langsmith-search'

In [11]:
tools = [wiki , arxiv , retriever_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/mahmoud/anaconda3/envs/lunfgraph/lib/python3.9/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=2000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=2000)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7fbd8ba14430>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbcaaec3

In [13]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Access keys
groq_api_key = os.getenv('GROQ_API_KEY')

In [14]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key ,model_name="Llama3-8b-8192")

In [15]:
llm.invoke("Hello")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.021666667, 'prompt_time': 0.001195475, 'queue_time': 0.0016524040000000001, 'total_time': 0.022862142}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-71cc16af-b8ce-47bb-a79f-f8f8f962d03f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

# prompt template

In [16]:
from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/mahmoud/anaconda3/envs/lunfgraph/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

# Agents 

In [17]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm ,tools , prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [18]:
## agent Executer
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.agents import AgentExecutor

memory = ConversationBufferMemory(k=3)  # Default memory setup


agent = AgentExecutor(agent=agent, tools=tools, memory=memory,verbose=True)

agent

/tmp/ipykernel_415126/563566781.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(k=3)  # Default memory setup


AgentExecutor(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[])), verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AI

In [19]:
agent.invoke({"input":"What is the capital of France?"})  



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'What is the capital of France?'}`


Page: Closed-ended question
Summary: A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.
A closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.

Paris

> Finished chain.


{'input': 'What is the capital of France?', 'history': '', 'output': 'Paris'}

In [21]:
agent.invoke({"input":"What is Machine Learning?"})  



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. 
From a theoretical viewpoint, p

{'input': 'What is Machine Learning?',
 'history': 'Human: What is the capital of France?\nAI: Paris',
 'output': "Machine learning (ML) is a subfield of artificial intelligence (AI) that involves the development and study of algorithms that can learn from data and improve their performance on a task without being explicitly programmed. It's a type of statistical learning that allows systems to automatically improve their performance on a task by learning from data."}